Variable Coefficients
===
$\DeclareMathOperator{\opdiv}{div}$

A setup with varying heat-conductivity is modeled by the equation

$$
-\opdiv \lambda(x) \nabla u(x) = f(x),
$$

where $\lambda$ is the heat-conductivity, relating the heat-flux 

$$
q = -\lambda \nabla u
$$

to the temperature gradient $\nabla u$.

In the case of discontinuous $\lambda$, the equation is understood in distributional sense. This includes the interface conditions: The temperature on the left and right side are equal, and the heat flux leaving the left side enters the right side:

\begin{eqnarray*}
u_l & = & u_r \\
\lambda_l \frac{\partial u_r}{\partial n} & = & \lambda_r \frac{\partial u_r}{\partial n}
\end{eqnarray*}

The variational form is: find $u \in H^1(\Omega)$

$$
\int_\Omega \lambda(x) \nabla u \nabla v = \int_\Omega f v
$$

There is no issue with discontinuous coefficients. Both interface conditions are included: 
* continuity of temperature $u$ by the continuity of the trial space
* continuity of the heat flux in weak sense, such as Neumann boundary conditions

In [ ]:
import netgen.gui
from ngsolve import *

Make a 2D geometry:

In [ ]:
from netgen.geom2d import *
geo = SplineGeometry()
geo.AddRectangle( (0,0), (1,1), leftdomain=1, rightdomain=0, 
                 bcs=['b','r','t','l'])
geo.AddCircle( (0.3,0.7), 0.1, leftdomain=2, rightdomain=1)
geo.AddRectangle ( (0.2,0.2), (0.9,0.3), leftdomain=3, rightdomain=1)
geo.SetMaterial(1, "air")
geo.SetMaterial(2, "source")
geo.SetMaterial(3, "bar")

mesh = Mesh(geo.GenerateMesh(maxh=0.03))
mesh.Curve(3)
Draw (mesh)

In [ ]:
print (mesh.GetMaterials())

In [ ]:
print (mesh.GetBoundaries())

In [ ]:
fes = H1(mesh, order=3, dirichlet="b|r")
u = fes.TrialFunction()
v = fes.TestFunction()

make a coefficient function taking one constant per material. Uses Python list comprehension.

In [ ]:
lamvalues = { "air" : 1, "bar" : 100, "source" : 2 }
lam = CoefficientFunction( 
    [lamvalues[mat] for mat in mesh.GetMaterials()])
Draw (lam, mesh, "lambda")

In [ ]:
lamvalues

use conductivity in bilinear-form:

In [ ]:
a = BilinearForm(fes)
a += lam*grad(u)*grad(v)*dx

f = LinearForm(fes)
f += 1*v*dx("source")
a.Assemble()
f.Assemble()

In [ ]:
reg = mesh.Materials("source|bar")
print (reg.VB(), reg.Mask())

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu, mesh, "temperature")
Draw (grad(gfu), mesh, "gradient")
Draw (-lam*grad(gfu), mesh, "heatflux")